Tic Tac Toe Spielstanderkenner

Ziel dieser Arbeit ist es, dass Bilder eines Tic Tac Toe Spiels eingegeben werden und der Spielstand automatisch erkannt wird. Wenn es einen Gewinner gibt, soll dieser Markiert und ausgegeben werden

Schritt 1 Bilder einlesen

In [ ]:
import os
import glob
from skimage import io

# Pfad zu eurem Bilder-Ordner
image_folder = "data/boards/"
# unterstützte Formate
extensions = ("*.jpg", "*.jpeg", "*.png", "*.bmp")

# Liste aller Bild-Pfade
paths = []
for ext in extensions:
    paths.extend(glob.glob(os.path.join(image_folder, ext)))

# Optional: sortieren nach Datum oder Nummer im Dateinamen
paths = sorted(paths)

print(f"Gefundene Bilder: {len(paths)}")


Schritt zwei Processing Pipeline je Bild

In [ ]:
from skimage import color, filters, morphology, transform, feature
import numpy as np

def preprocess(img):
    """Graustufen, Glätten, Binarisierung, Rauschen entfernen."""
    gray    = color.rgb2gray(img)
    blurred = filters.gaussian(gray, sigma=1.0)
    thresh  = filters.threshold_otsu(blurred)
    binary  = blurred > thresh
    clean   = morphology.remove_small_objects(binary, min_size=150)
    return clean

def find_grid(binary):
    """Kanten → Hough-Linien → vier horizontale + vier vertikale Linien clustern → Ecken."""
    edges = feature.canny(binary, sigma=2)
    # TODO: HoughLinesP implementieren, cluster auf 4+4 Linien
    # return corners als Array shape (4,4,2) oder Liste von 16 (x,y)-Paaren
    raise NotImplementedError

def warp_board(img, corners):
    """Perspektivische Entzerrung auf quadratisches Board (z.B. 300×300 px)."""
    src = np.array(corners, dtype=np.float32)
    dst = np.array([[0,0], [300,0], [300,300], [0,300]], dtype=np.float32)
    tform = transform.ProjectiveTransform()
    tform.estimate(src, dst)
    board = transform.warp(img, tform)
    return board

def classify_cell(cell):
    """Erkenne X, O oder leer in einer 100×100-Zelle."""
    # Kreis-Detektion (HoughCircles) → "O"
    # Linien-Detektion (HoughLines) → "X"
    # Sonst: None
    raise NotImplementedError

def process_image(path):
    # 1. Einlesen und in Float [0,1] umwandeln
    img_rgb = io.imread(path)
    if img_rgb.dtype != np.float64:
        img = img_rgb / 255.0
    else:
        img = img_rgb

    # 2. Preprocessing + Grid finden
    binary = preprocess(img)
    corners = find_grid(binary)

    # 3. Warp & Zellen extrahieren
    board = warp_board(img, corners)
    cell_size = board.shape[0] // 3

    grid = []
    for i in range(3):
        row = []
        for j in range(3):
            cell = board[
                i*cell_size:(i+1)*cell_size,
                j*cell_size:(j+1)*cell_size
            ]
            row.append(classify_cell(cell))
        grid.append(row)

    return grid


Schleife für alle Bilder

In [ ]:
results = {}
for path in paths:
    try:
        grid = process_image(path)
        results[os.path.basename(path)] = grid
        print(f"{os.path.basename(path)} → {grid}")
    except Exception as e:
        print(f"Fehler bei {path}: {e}")


Mögliche Vorschläge:

4. Empfehlungen
Modularisieren: Packt jede Funktion in ein eigenes Modul (preprocess.py, grid.py, warp.py, recognition.py), dann importiert ihr sie in main.py.

Parameter-Tuning: Führt Tests mit verschiedenen sigma, min_size und Hough-Schwellen durch und protokolliert, was am besten funktioniert.

Fehlerbehandlung: Wenn find_grid mal nicht genügend Linien findet, überspringt das Bild oder versucht einen alternativen Ansatz (z. B. Kanten-Dilatation).

Batch-Logging: Schreibt in eine Log-Datei, welche Bilder erfolgreich geprocessed wurden und welche fehlschlugen.